In [1]:
import numpy as np
import os.path
import shutil, os
import sys#TylerK: Add sys to get cmdline_helper from NRPy top directory; remove this line and next when debugged
sys.path.append('../')

import Schwarzschild.NQC_corrections as nqc
import Schwarzschild.nqc_interp as nqi

from Schwarzschild.constant_coeffs import compute_const_coeffs


with open('Schwarzschild/SymPy_Hreal_Sph_on_bottom.txt', 'w') as output:
    for line in reversed(list(open("Schwarzschild/Hamiltonian-Sph_Hreal_one_line_expressions-Debug.txt"))):
        output.write("%s\n" % line.rstrip())

with open('Schwarzschild/SymPy_Hreal_Cart_on_bottom.txt', 'w') as output:
    for line in reversed(list(open("Schwarzschild/Hamiltonian-Cart_Hreal_one_line_expressions-Debug.txt"))):
        output.write("%s\n" % line.rstrip())
        
import Schwarzschild.Sph_Hamiltonian_and_derivs_playground as Hadsph
Hadsph.output_H_and_derivs()

import Schwarzschild.Cart_Hamiltonian_and_derivs_playground as Hadcrt
Hadcrt.output_H_and_derivs()

import Schwarzschild.Sph_Hamiltonian_second_derivs_playground as Hsdsph
Hsdsph.output_H_sec_derivs()

import Schwarzschild.Cart_Hamiltonian_second_derivs_playground as Hsdcrt
Hsdcrt.output_H_sec_derivs()

In [2]:
from Playground_Pycodes.d2Hdr2 import compute_d2Hdr2
from Playground_Pycodes.d2Hdrdpphi import compute_d2Hdrdpphi
from Playground_Pycodes.new_dHdp2 import new_compute_dHdp2 # For testing
from Playground_Pycodes.new_dHdp3 import new_compute_dHdp3 # For testing
from Playground_Pycodes.d2Hdx2 import compute_d2Hdx2 # For testing
from Playground_Pycodes.d2Hdxdp2 import compute_d2Hdxdp2 # For testing
from Playground_Pycodes.d2Hdxdp3 import compute_d2Hdxdp3 # For testing
from Playground_Pycodes.d2Hdp22 import compute_d2Hdp22 # For testing
from Playground_Pycodes.d2Hdp2dp3 import compute_d2Hdp2dp3 # For testing
from Playground_Pycodes.d2Hdp32 import compute_d2Hdp32 # For testing

In [8]:
m1 = 23.
m2 = 10.

S1_Sph = np.array([0.01, 0.03, 0.02])
S2_Sph = np.array([0.04, -0.06, -0.05])

S1_Cart = np.array([0.01, 0.02, -0.03])
S2_Cart = np.array([0.04, -0.05, 0.06])


EMgamma = 0.577215664901532860606512090082402431

S1_Sph *= m1*m1
S2_Sph *= m2*m2
S1_Cart *= m1*m1
S2_Cart *= m2*m2

# Binary system total mass $M$
M = m1 + m2

# Inverse mass terms used repeatedly when computing initial conditions
Minv = np.divide(1,M)
Msqinv = Minv*Minv

# Symmetric mass ratio $\eta$
eta = m1*m2*Msqinv

S1hat_Sph = Msqinv*S1_Sph
S2hat_Sph = Msqinv*S2_Sph
SKerr_Sph = np.add(S1_Sph, S2_Sph)
S1hat_Cart = Msqinv*S1_Cart
S2hat_Cart = Msqinv*S2_Cart
SKerr_Cart = np.add(S1_Cart, S2_Cart)

# Normalize S_Kerr by total mass
SKerr_Sph *= Msqinv
SKerr_Cart *= Msqinv

# Compute a, which is a parameter in metric potentials of a Kerr spacetime
# See https://arxiv.org/abs/0912.3517 Equation (4.9)
asq_Cart = np.dot(SKerr_Cart,SKerr_Cart)
a_Cart = np.sqrt(asq_Cart)
asq_Sph = np.dot(SKerr_Sph,SKerr_Sph)
a_Sph = np.sqrt(asq_Sph)

print("a_Sph = %.15e" % a_Sph)
print("a_Cart = %.15e" % a_Cart)

if np.abs(a_Sph - a_Cart) < 1e-15:
    print("a = %.15e" % a_Sph)
    a = a_Sph


KK, k0, k1, k2, k3, k4, k5, k5l, dSO, dSS = compute_const_coeffs(eta,EMgamma,a)


r =  2.129621188966642e+01
theta = np.pi/2
phi = 0.

qsph = [r,theta,phi]
qcart = [r,0,0]

pr = 0.00000000e+00 
ptheta = -5.63544577e-05  
pphi = 4.97363575e+00

psph = [pr,ptheta,pphi]
pcart = [pr,pphi/r,-ptheta/r]

tort = 5

dHdp2 = new_compute_dHdp2(m1, m2, eta, qcart[0], qcart[1], qcart[2],
                        pcart[0], pcart[1], pcart[2],
                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
dHdp3 = new_compute_dHdp3(m1, m2, eta, qcart[0], qcart[1], qcart[2],
                        pcart[0], pcart[1], pcart[2],
                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
#d2Hdx2 = compute_d2Hdx2(m1, m2, eta, qcart[0], qcart[1], qcart[2],
#                        pcart[0], pcart[1], pcart[2],
#                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
#                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
#d2Hdxdp2 = compute_d2Hdxdp2(m1, m2, eta, qcart[0], qcart[1], qcart[2],
#                        pcart[0], pcart[1], pcart[2],
#                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
#                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
#d2Hdxdp3 = compute_d2Hdxdp3(m1, m2, eta, qcart[0], qcart[1], qcart[2],
#                        pcart[0], pcart[1], pcart[2],
#                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
#                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
d2Hdp22 = compute_d2Hdp22(m1, m2, eta, qcart[0], qcart[1], qcart[2],
                        pcart[0], pcart[1], pcart[2],
                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)

#d2Hdp2dp3 = compute_d2Hdp2dp3(m1, m2, eta, qcart[0], qcart[1], qcart[2],
#                        pcart[0], pcart[1], pcart[2],
#                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
#                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)
d2Hdp32 = compute_d2Hdp32(m1, m2, eta, qcart[0], qcart[1], qcart[2],
                        pcart[0], pcart[1], pcart[2],
                        S1hat_Cart[0], S1hat_Cart[1], S1hat_Cart[2],
                        S2hat_Cart[0], S2hat_Cart[1], S2hat_Cart[2], KK, k0, k1, dSO, dSS, tort, EMgamma)

#print("d2Hdx2 = %.15e" % (d2Hdx2[1]/eta))
#print("d2Hdxdpy = %.15e" % (d2Hdxdp2[1]/eta))
#print("d2Hdxdpz = %.15e" % (d2Hdxdp3[1]/eta))
print("d2Hdpy2 = %.15e" % (d2Hdp22[1]/eta))
#print("d2Hdpydpz = %.15e" % (d2Hdp2dp3[1]/eta))
print("d2Hdpz2 = %.15e" % (d2Hdp32[1]/eta))

u = 1/r
u2 = u*u
u3 = u2*u
u4 = u3*u

term1a = psph[1]*d2Hdp32[1]*u4
term1b = 0 #- 2*psph[1]*psph[2]*d2Hdp2dp3[1]*u4
term1c = psph[2]*d2Hdp22[1]*u4
term1 = term1a + term1b + term1c

term2a = 2*psph[2]*dHdp2*u3
term2b = - 2*psph[1]*dHdp3*u3
term2 = term2a + term2b

term3a = 0 #2*psph[1]*d2Hdxdp3[1]*u2
term3b = 0 #- 2*psph[2]*d2Hdxdp2[1]*u2
term3 = term3a + term3b

term4 = 0 #d2Hdx2[1]

d2Hdr2_from_cart = term1 + term2 + term3 + term4

d2Hdr2 = compute_d2Hdr2(m1, m2, eta, qsph[0], qsph[1], qsph[2],
                        psph[0], psph[1], psph[2],
                        S1hat_Sph[0], S1hat_Sph[1], S1hat_Sph[2],
                        S2hat_Sph[0], S2hat_Sph[1], S2hat_Sph[2], KK, k0, k1, dSO, dSS, tort, EMgamma)

print("d2Hdr2 = %.16e"%(d2Hdr2[1]/eta))
print("d2Hdr2_from_cart = %.16e"%(d2Hdr2_from_cart/eta))
print('\n')

print('term1: %.15e'% (term1/eta))
print('\t term1a: %.15e'% (term1a/eta))
print('\t term1b: %.15e'% (term1b/eta))
print('\t term1c: %.15e'% (term1c/eta))

print('term2: %.15e'% (term2/eta))
print('\t term2a: %.15e'% (term2a/eta))
print('\t term2b: %.15e'% (term2b/eta))

print('term3: %.15e'% (term3/eta))
print('\t term3a: %.15e'% (term3a/eta))
print('\t term3b: %.15e'% (term3b/eta))

print('term4: %.15e'% (term4/eta))

print('\n')


a_Sph = 1.346006875336737e-02
a_Cart = 1.346006875336737e-02
a = 1.346006875336737e-02
d2Hdpy2 = 9.469565217391304e+00
d2Hdpz2 = 9.469565217391304e+00
d2Hdr2 = 3.4165659579972474e-03
d2Hdr2_from_cart = 2.5066864769599465e-03


term1: 2.289758382951149e-04
	 term1a: -2.594471338909250e-09
	 term1b: 0.000000000000000e+00
	 term1c: 2.289784327664538e-04
term2: 2.277710638664832e-03
	 term2a: 2.277710638372412e-03
	 term2b: 2.924200506448474e-13
term3: 0.000000000000000e+00
	 term3a: 0.000000000000000e+00
	 term3b: 0.000000000000000e+00
term4: 0.000000000000000e+00




In [4]:
#print("H from d2Hdx2 = %.16e" % d2Hdx2[0])
#print("H from d2Hdxdp2 = %.16e" % d2Hdxdp2[0])
#print("H from d2Hdxdp3 = %.16e" % d2Hdxdp3[0])
print("H from d2Hdp22 = %.16e" % d2Hdp22[0])
#print("H from d2Hdp2dp3 = %.16e" % d2Hdp2dp3[0])
print("H from d2Hdp32 = %.16e" % d2Hdp32[0])
print("H from d2Hdr2 = %.16e" % d2Hdr2[0])


H from d2Hdp22 = 5.4543529005170156e-02
H from d2Hdp32 = 5.4543529005170156e-02
H from d2Hdr2 = 5.4543529005170149e-02


In [5]:

#d2Hdrdpphi = compute_d2Hdrdpphi(m1, m2, eta, q[0], q[1], q[2],
#                        p[0], p[1], p[2],
#                        S1[0], S1[1], S1[2],
#                        S2[0], S2[1], S2[2], KK, k0, k1, dSO, dSS, 1, EMgamma)

#print(d2Hdrdpphi)



In [6]:
#for i in range(6):
#    if i < 3:
#        print("sphValues["+str(i)+"] = "+str(q[i])+";")
#    elif i < 6:
#        print("sphValues["+str(i)+"] = "+str(p[i-3])+";")


#print("sphValues[6] = 5.290000000000000e+00;")
#print("sphValues[7] = 1.057999822554391e+01;")
#print("sphValues[8] = -1.587000118297058e+01;")
#print("sphValues[9] = 4.000000000000000e+00;")
#print("sphValues[10] = -4.999999329128126e+00;")
#print("sphValues[11] = 6.000000559059830e+00;")